# Computing vertex amplitudes with Julia

In this notebook we compute the vertex amplitudes. We store them in multiple formats, so that they can be retrieved with Python and julia 

In [1]:
using SL2Cfoam, HalfIntegers, JLD2, NPZ

In [2]:
using JupyterFormatter
enable_autoformat()

1-element Vector{Function}:
 format_current_cell (generic function with 1 method)

In [3]:
# Barbero-Immirzi parameter
γ = 1.0

# initializing sl2cfoam-next   
sl2c_data_folder = "/home/frisus/Scrivania/sl2cfoam-next-dev/data_sl2cfoam"
sl2c_configuration = SL2Cfoam.Config(VerbosityOff, VeryHighAccuracy, 100, 0)
sl2c_result_return = (ret = true, store = true, store_batches = true)
# to store only the booster functions and not the full amplitude, use:
# sl2c_result_return = (ret = true, store = false, store_batches = false);

SL2Cfoam.cinit(sl2c_data_folder, γ, sl2c_configuration)

In [4]:
# folder in which vertices are stored
VERTEX_BF_FOLDER = "../../data/BF_vertices"
VERTEX_EPRL_FOLDER = "../../data/EPRL_vertices"

# max value of boundary spins
J_MAX = 6

# truncation parameter for EPRL
DL = 0;

In [11]:
function BFVerticesCompute(J_MAX, VERTEX_FOLDER, step = half(1))

    mkpath("$(VERTEX_FOLDER)/Julia")
    mkpath("$(VERTEX_FOLDER)/Python")

    for j = half(1):step:J_MAX

        j_float = twice(j) / 2

        spins = [j for index = 1:10]

        v = vertex_BF_compute(spins)
        vertex = v.a

        @save "$(VERTEX_FOLDER)/Julia/vertex_j_$(j_float).jld2" vertex
        npzwrite("$(VERTEX_FOLDER)/Python/vertex_j_$(j_float).npz", vertex)

    end

end

computing_BF_vertices (generic function with 2 methods)

In [12]:
@time BFVerticesCompute(J_MAX, VERTEX_BF_FOLDER)

  4.965920 seconds (4.40 M allocations: 217.641 MiB, 1.26% gc time, 92.61% compilation time)


In [13]:
function EPRLVerticesCompute(J_MAX, DL, VERTEX_FOLDER, step = half(1))

    mkpath("$(VERTEX_FOLDER)/Julia/Dl_$(DL)")
    mkpath("$(VERTEX_FOLDER)/Python/Dl_$(DL)")

    for j = half(1):step:J_MAX

        j_float = twice(j) / 2

        spins = [j for index = 1:10]

        v = vertex_compute(spins, DL; result = sl2c_result_return)
        vertex = v.a

        @save "$(VERTEX_FOLDER)/Julia/Dl_$(DL)/vertex_j_$(j_float).jld2" vertex
        npzwrite("$(VERTEX_FOLDER)/Python/Dl_$(DL)/vertex_j_$(j_float).npz", vertex)

    end

end

computing_EPRL_vertices (generic function with 2 methods)

In [14]:
@time EPRLVerticesCompute(J_MAX, DL, VERTEX_EPRL_FOLDER)

  1.862917 seconds (2.70 M allocations: 94.089 MiB, 0.86% gc time, 6.58% compilation time)


# Loading amplitudes in julia

In [22]:
function EPRLVertexLoadJulia(J, DL, VERTEX_FOLDER)

    j_float = twice(J) / 2

    spins = [j_float for index = 1:10]

    @load "$(VERTEX_FOLDER)/Julia/Dl_$(DL)/vertex_j_$(j_float).jld2" vertex

    return vertex

end

EPRLVertexLoadJulia (generic function with 1 method)

In [25]:
function EPRLVertexFromJuliaToPython(J_MAX, DL, VERTEX_FOLDER, step = half(1))

    mkpath("$(VERTEX_FOLDER)/Python/Dl_$(DL)")

    for j = half(1):step:J_MAX

        j_float = twice(j) / 2

        @load "$(VERTEX_FOLDER)/Julia/Dl_$(DL)/vertex_j_$(j_float).jld2" vertex

        npzwrite("$(VERTEX_FOLDER)/Python/Dl_$(DL)/vertex_j_$(j_float).npz", vertex)

    end

end

EPRLVertexFromJuliaToPython (generic function with 2 methods)

In [27]:
EPRLVertexFromJuliaToPython(J_MAX, 20, VERTEX_EPRL_FOLDER)